## Agile Hardware Design
***
# Hello Chisel

<img src="./images/chisel_logo.svg" alt="scala logo" style="float:right; width: 200px;"/>

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)

## Before we started. We test UCB Chisel Structure and make sure it worked.

In [1]:
// Before we start. We test for UCB stcture and make sure they worked.
// Below, we test for UCB stcture and make sure they worked.
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/002_HelloChisel/001_Code/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import dotvisualizer._

In [3]:
// Chisel Code: Declare a new module definition
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code; it is very complicated Scala
println(getVerilog(new Passthrough))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd2.sc 6:10]
endmodule



defined class Passthrough

In [4]:
// Chisel Code: Declare a new module definition
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

// Chisel Code, but pass in a parameter to set widths of ports
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}
// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code; it is very complicated Scala
println(getVerilog(new Passthrough))

// Let's now generate modules with different widths
println(getVerilog(new PassthroughGenerator(10)))
println(getVerilog(new PassthroughGenerator(20)))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd3.sc 6:10]
endmodule

Elaborating design...
Done elaborating.
module PassthroughGenerator(
  input        clock,
  input        reset,
  input  [9:0] io_in,
  output [9:0] io_out
);
  assign io_out = io_in; // @[cmd3.sc 15:10]
endmodule

Elaborating design...
Done elaborating.
module PassthroughGenerator(
  input         clock,
  input         reset,
  input  [19:0] io_in,
  output [19:0] io_out
);
  assign io_out = io_in; // @[cmd3.sc 15:10]
endmodule



defined class Passthrough
defined class PassthroughGenerator

## Play With Today's Slides

### [https://github.com/agile-hw/lectures](https://github.com/agile-hw/lectures)
* Can work with it directly in browser via [Binder](https://mybinder.org/v2/gh/agile-hw/lectures/HEAD?urlpath=tree)
* Can clone and run Jupyter locally
  * Will need to install additional things for presentation, Scala, and Chisel
  * Repo contains `install.sh` to help with this

## Plan for Today

* Brief taste of Scala
* Brief taste of Chisel
* _Close the loop:_ build & test simple Chisel module

## Scala Language Summary

<img src="./images/scala-spiral.png" alt="scala logo" style="float:right; width: 100px;"/>

_**Language Features**_
* Object oriented with strong static type system
* Native support for functional programing
* Runs on top of the JVM (and can interoperate with Java binaries)
* Tries to catch many potential errors at compile time

_**Rationale for Using Scala**_
* Great support for implementing embedded domain-specific languages (DSL), e.g. Chisel
* Object oriented and functional features help make great _generators_
* Type system and included standard library’s collections

## Scala Execution Mechanisms

_**Standard Compilation -> Execution**_
* Compile Scala program (as Java bytecode) and run on JVM
* Code needs to be structured in classes and have a `main`
* Typically use appropriate build tool (e.g. sbt) or an IDE (e.g. IntelliJ)

_**Read-Eval-Print Loop (REPL)**_
* Can launch a REPL (interpreter) and write (then evaluate) a single line at a time
* Great for testing out features
* The Jupyter notebooks in this course (where we can execute snippets in isolation) are built on an extension to this ([Almond](https://almond.sh))

## Scala Literals

* Common Simple Scala Types - `Int`, `Float`, `Long`, `Double`, `Byte`, `Char`, `String`
* _Syntax Note:_ semicolons are optional and rarely used

In [5]:
2 + 3
5.0 / 2
"hello"

res4_0: Int = 5
res4_1: Double = 2.5
res4_2: String = "hello"

## Scala Type Inference

* Everything is an object, even simple types
* Types must be known/resolved at compile time (_statically typed_)
* Scala has _type inference_, so can often omit type specifier
* _Syntax Note:_ unlike C/Java, type goes after name (instead of before)

In [6]:
4
4: Int
4: Float
4: Double
4: Char
4.toFloat

res5_0: Int = 4
res5_1: Int = 4
res5_2: Float = 4.0F
res5_3: Double = 4.0
res5_4: Char = '\u0004'
res5_5: Float = 4.0F

## Declaring Scala Variables

### `var` - **Mutable** variable (_discouraged_)
* Can reassign, like conventional languages

### `val` - **Immutable** variable (_encouraged_)
* Enables write-once semantics common to many functional languages
* Allows compiler to safely perform more ambitious optimizations
* Can increase code clarity by renaming values each step of the way

In [7]:
var mutX = 0
mutX = 2
val constX = 42

mutX: Int = 2
constX: Int = 42

## Chisel Is Embedded in Scala

* Chisel designs are Scala programs (i.e. everything we write in this course is Scala)
* A Chisel design is simply a Scala program that makes use of the Chisel library
* Thanks to Scala language features, using Chisel library operations often feels like a full-fledged language
  * Operator overloading and concise syntax

## Chisel Tool Flow (Frontend)

<img src="images/frontend.svg" alt="Chisel frontend" style="width:80%;margin:auto"/>

* The generated Circuit (`.fir` file) is a specific design instance, and it can be passed off to a _backend_ for simulation or implementation

## Loading The Chisel Library Into a Notebook

In [8]:
// UCB path
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))
//println("UCSC:")
//val path1 = System.getProperty("user.dir") + "/resource/chisel_deps.sc"
//println ("path1:" + path1)
//interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path1)))

path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/002_HelloChisel/001_Code/source/load-ivy.sc"

In [9]:
/* import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test
*/
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import dotvisualizer._

In [10]:
// Chisel Code: Declare a new module definition
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

defined class Passthrough

In [11]:
// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code; it is very complicated Scala
println(getVerilog(new Passthrough))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd9.sc 6:10]
endmodule



## Simple Chisel Types

* Scala literals must be cast to Chisel

### `Bool` - single-bit logic signal (`.B`)
* Do not confuse with Scala's `Boolean`

### `UInt` - unsigned integer (`.U`)
* Bitwidth set explicitly or inferred

### `SInt` - signed integer (`.S`)
* Operates like `UInt` but signed


In [12]:
0.B
true
true.B
val myBool: Bool = true.B

6
6.U
6.U(8.W)
val myUInt: UInt = 4.U
val myUInt8 = 4.U(8.W)

-2
-2.S

res11_0: Bool = Bool(false)
res11_1: Boolean = true
res11_2: Bool = Bool(true)
myBool: Bool = Bool(true)
res11_4: Int = 6
res11_5: UInt = UInt<3>(6)
res11_6: UInt = UInt<8>(6)
myUInt: UInt = UInt<3>(4)
myUInt8: UInt = UInt<8>(4)
res11_9: Int = -2
res11_10: SInt = SInt<2>(-2)

## Chisel Operators (Brief Intro)

* Most of the operators you would expect exist
    * Some of them have different symbols (e.g. `===` for equality test)
    * Pay attention to result widths
    * Consult [Chisel Cheat Sheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/download/3.6.0/chisel_cheatsheet.pdf) for brief summary
* _**Logical:**_ `!`, `&&`, `||`
* _**Arithmetic:**_ `+`, `-`, `*`, `/`, `%`
* _**Bitwise:**_ `~`, `&`, `|`, `^`
* _**Relational:**_ `===`, `=/=`, `<`, `<=`, `>`, `>=`
* _**Shifts:**_ `<<`, `>>`
* _**Others:**_ extraction, fill, concatenation, mux, reductions


In [13]:
val a = 7.U
val b = 5.U

// Not expected to work (needs to be inside Module)
// a + b

a: UInt = UInt<3>(7)
b: UInt = UInt<3>(5)

## First Chisel Module

In [14]:
class MyXOR extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Output(Bool())
    })
    io.c := io.a ^ io.b
    //io.c := io.a
}

defined class MyXOR

<img src="images/myxor.svg" alt="MyXOR schematic"  style="width:80%;margin:auto"/>

## Looking At Generated Design

In [15]:
//printVerilog(new MyXOR)
println(getVerilog(new MyXOR))

Elaborating design...
Done elaborating.
module MyXOR(
  input   clock,
  input   reset,
  input   io_a,
  input   io_b,
  output  io_c
);
  assign io_c = io_a ^ io_b; // @[cmd13.sc 7:18]
endmodule



In [16]:
// Chisel Code: Declare a new module definition
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

defined class Passthrough

In [17]:
println(getVerilog(new Passthrough))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd15.sc 6:10]
endmodule



In [18]:
// Chisel Code, but pass in a parameter to set widths of ports
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}
// Let's now generate modules with different widths
println(getVerilog(new PassthroughGenerator(10)))
println(getVerilog(new PassthroughGenerator(20)))

Elaborating design...
Done elaborating.
module PassthroughGenerator(
  input        clock,
  input        reset,
  input  [9:0] io_in,
  output [9:0] io_out
);
  assign io_out = io_in; // @[cmd17.sc 6:10]
endmodule

Elaborating design...
Done elaborating.
module PassthroughGenerator(
  input         clock,
  input         reset,
  input  [19:0] io_in,
  output [19:0] io_out
);
  assign io_out = io_in; // @[cmd17.sc 6:10]
endmodule



defined class PassthroughGenerator

In [19]:
visualize(() => new MyXOR)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: MyXOR Pages: 1 -->
 
 
 MyXOR 
 
 
 cluster_MyXOR 
 
 MyXOR 
 
<!-- cluster_MyXOR_clock -->
 
 cluster_MyXOR_clock 
 
 clock 
 
<!-- cluster_MyXOR_reset -->
 
 cluster_MyXOR_reset 
 
 reset 
 
<!-- cluster_MyXOR_io_a -->
 
 cluster_MyXOR_io_a 
 
 io_a 
 
<!-- op_xor_1 -->
 
 op_xor_1 
 
 
 ● 
 
 xor 
 
 ● 
 
 ● 
 
<!-- cluster_MyXOR_io_a->op_xor_1 -->
 
 cluster_MyXOR_io_a->op_xor_1:in1 
 
 
 
<!-- cluster_MyXOR_io_b -->
 
 cluster_MyXOR_io_b 
 
 io_b 
 
<!-- cluster_MyXOR_io_b->op_xor_1 -->
 
 cluster_MyXOR_io_b->op_xor_1:in2 
 
 
 
<!-- cluster_MyXOR_io_c -->
 
 cluster_MyXOR_io_c 
 
 io_c 
 
<!-- op_xor_1->cluster_MyXOR_io_c -->
 
 op_xor_1:out->cluster_MyXOR_io_c

## Chisel Tool Flow (Backend)
<p><p>

<img src="images/backend.svg" alt="Chisel backend" style="width:75%;margin:auto"/>

## Brief ChiselTest Intro

* Can write a Scala program to interact with simulation of elaborated design
* Can set inputs and look at outputs
* Can use full power of Scala to generate test inputs and outputs to compare with
* Will continue to cover more features in coming lectures

### `poke` - set value of wire

### `peek` - read value of wire

### `expect` - read value and compare (assert)

## ChiselTest Execution

In [20]:
test(new MyXOR()) { x =>
    x.io.a.poke(0.B)
    x.io.b.poke(0.B)
    x.io.c.expect(0.B)  // 0 ^ 0

    x.io.a.poke(0.B)
    x.io.b.poke(1.B)
    x.io.c.expect(1.B)  // 0 ^ 1

    x.io.a.poke(1.B)
    x.io.b.poke(0.B)
    x.io.c.expect(1.B)  // 1 ^ 0

    x.io.a.poke(1.B)
    x.io.b.poke(1.B)
    x.io.c.expect(0.B)  // 1 ^ 1
}

Elaborating design...
Done elaborating.
test MyXOR Success: 0 tests passed in 2 cycles in 0.032805 seconds 60.97 Hz


## Chisel Versioning

<img src="./images/chisel_logo.svg" alt="scala logo" style="float:right; width: 200px;"/>

* This quarter, we will be using **Chisel 3.6** for lecture, labs, and homework
  * You are welcome to use a newer Chisel for the project
  * _Rationale:_ Chisel is undergoing big changes, and not quite stabilized
* Provided code and environments will correctly pull in the right version
  * When reading documentation, be aware may be describing a feature in newer version
* Relevant Chisel versions
  * 3.6 (April 2023) is the bridge from the old to the new
  * 5.1 (November 2023) is the most recent released version
  * 6 is under public development
  * There is no version 4, as it was skipped to make version jump apparent